In [1]:
import pandas as pd
import numpy as np
import os
import ftfy

In [2]:
df = pd.read_csv('Data/player_stats.csv')
df.head()

,id,name,dateOfBirth,Age,Height,Foot,Position,OtherPosition,National,MarketValue,...,25MP,25AP,Ranking,2020AvgMV,2021AvgMV,2022AvgMV,2023AvgMV,2024AvgMV,2025AvgMV,TotalCups
0,591949,Aaron Hickey,"Jun 10, 2002",22.0,"1,85m",both,Right-Back,"['Left-Back', 'Left Midfield']",Scotland,22000000.0,...,0.0,0,432.000,1.845000e+06,8.666667e+06,17000000.0,2.833333e+07,2.500000e+07,0.0,0
1,434207,Aaron Connolly,"Jan 28, 2000",25.0,"1,74m",right,Centre-Forward,['Second Striker'],Ireland,2500000.0,...,505.0,14,3.644,5.500000e+06,7.000000e+06,5500000.0,3.500000e+06,2.500000e+06,0.0,2
2,578459,Aaron Bastiaans,"Apr 4, 2002",22.0,"1,84m",right,Left Winger,"['Right Winger', 'Centre-Forward']",Netherlands,0.0,...,0.0,0,0.000,1.500000e+05,1.250000e+05,175000.0,0.000000e+00,0.000000e+00,0.0,0
3,884244,AJ Marcucci,"Jul 31, 1999",25.0,"1,91m",NaN,Goalkeeper,[],United States,250000.0,...,0.0,0,0.000,0.000000e+00,1.000000e+05,125000.0,1.500000e+05,2.166667e+05,0.0,0
4,401362,AarÃ³n Herrera,"Jun 6, 1997",27.0,"1,80m",right,Right-Back,"['Left-Back', 'Right Midfield']",United States,2000000.0,...,0.0,0,4.274,9.666667e+05,1.600000e+06,2000000.0,2.000000e+06,2.000000e+06,0.0,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13405 entries, 0 to 13404
Data columns (total 64 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              13405 non-null  int64  
 1   name            13405 non-null  object 
 2   dateOfBirth     13389 non-null  object 
 3   Age             13389 non-null  float64
 4   Height          13209 non-null  object 
 5   Foot            12714 non-null  object 
 6   Position        13344 non-null  object 
 7   OtherPosition   13405 non-null  object 
 8   National        13166 non-null  object 
 9   MarketValue     13405 non-null  float64
 10  Outfitter       3075 non-null   object 
 11  Club_name       13389 non-null  object 
 12  ContractExpiry  13389 non-null  object 
 13  ContractOption  1344 non-null   object 
 14  20YC            13405 non-null  int64  
 15  202YC           13405 non-null  int64  
 16  20RC            13405 non-null  int64  
 17  20G             13405 non-null 

#### Handling missing values

In [4]:
#Check for misssing values in columns and store in df
missing = df.isnull().sum()
missing[missing > 0]

dateOfBirth          16
Age                  16
Height              196
Foot                691
Position             61
National            239
Outfitter         10330
Club_name            16
ContractExpiry       16
ContractOption    12061
dtype: int64

In [5]:
#Drop null columns
df.dropna(subset=['Height'], inplace=True)
df.dropna(subset=['Age'], inplace=True)
df.dropna(subset=['Club_name'], inplace=True)
df.dropna(subset=['Position'], inplace=True)
df.dropna(subset=['National'], inplace=True)


#Set foot as 'hand' for goalkeepers
df.loc[df['Position'] == 'Goalkeeper', 'Foot'] = 'hand'

#Find average values for foot using player position
foot_dict = df.groupby('Position')['Foot'].agg(lambda x: x.value_counts().index[0]).to_dict()
#Use the dictionary to fill in missing values
df['Foot'] = df.apply(lambda row: foot_dict[row['Position']] if pd.isnull(row['Foot']) else row['Foot'], axis=1)

missing = df.isnull().sum()
missing[missing > 0]

Outfitter          9894
ContractOption    11630
dtype: int64

#### Transforming data

In [6]:
#Fix player names
df['name'] = df['name'].apply(lambda x: ftfy.fix_text(x))
#Fix club names
df['Club_name'] = df['Club_name'].apply(lambda x: ftfy.fix_text(x))

#Convert age to int
df['Age'] = df['Age'].astype(int)

#Convert height to float
df['Height'] = df['Height'].str.replace(',', '.').str.replace('m', '').astype(float)

#### Save cleaned data

In [31]:
df.to_csv('Data/player_stats_cleaned.csv', index=False, header=True)